In [4]:
### Part 1
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEndpoint


In [5]:
import os 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
HuggingFaceApi = os.getenv('HF_TOKEN')
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEndpoint


engine = create_engine("sqlite:///AppleSales.db")
db = SQLDatabase(engine=engine)



llm = ChatGroq(model = "llama3-8b-8192",api_key=groq_api_key,
               temperature= 1, max_tokens=1024
)



from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
system_message = prompt_template.format(dialect="SQLite", top_k=5)

from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

c:\vs_code\Genai\genai_env\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [6]:
## Part 2

In [7]:
### Create 2nd tools to retreive question answer from the pdf

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate


pdf_data = PyPDFDirectoryLoader(r'C:\vs_code\Genai\pdf_directory').load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(pdf_data)


from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

from langchain_community.vectorstores import FAISS
vector_databse = FAISS.from_documents(text_splitter, embeddings)
retriever = vector_databse.as_retriever()

from langchain.tools.retriever import create_retriever_tool


prompt = ChatPromptTemplate.from_template(
    """
    answer the questions based on the provided context only.
    Please provide the most accurate  response based on the question 
    also make sure to give answer in simple english language 
    <context>
    {context}
    <context>
    Question:{input}
    
    """
        )

retriever_tool = create_retriever_tool(
    retriever,
    "PDF SEARCH",
    "Search Any Kind Of Information That Is Given  In The PDF Directory ",
    document_prompt= prompt
)

c:\vs_code\Genai\genai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
combined_tools = tools + [retriever_tool] 
combined_tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016742DE5E90>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016742DE5E90>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016742DE5E90>),
 QuerySQLCheckerTool(description='Use this tool to 

In [9]:
memory = MemorySaver()

agent_executor = create_react_agent(llm, combined_tools, messages_modifier= system_message,
                                    )

C:\Users\javar\AppData\Local\Temp\ipykernel_6212\1005626057.py:3: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, combined_tools, messages_modifier= system_message,


In [10]:
question = "Which State Is Having A Highest Sales?"
config = {"configurable": {"thread_id": "abc123"}}


for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    config = config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which State Is Having A Highest Sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_dxc6)
 Call ID: call_dxc6
  Args:
    tool_input: 
  sql_db_schema (call_wfc7)
 Call ID: call_wfc7
  Args:
    table_names:
================================= Tool Message =================================
Name: sql_db_schema

Error: table_names {''} not found in database
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_n579)
 Call ID: call_n579
  Args:
    table_names: AppleSales
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "AppleSales" (
	"State" TEXT, 
	"Region" TEXT, 
	"iPhone Sales (in million units)" FLOAT, 
	"iPad Sales (in million units)" FLOAT, 
	"Mac Sales (in million units)" FLOAT, 
	"Wearables (in millio

In [14]:

question = "Which State Is Having A Highest Sales?"

response  = agent_executor.invoke(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    
    config = config
        )

In [15]:
response

{'messages': [HumanMessage(content='Which State Is Having A Highest Sales?', additional_kwargs={}, response_metadata={}, id='906622ce-368c-4566-9ded-573ee4ea0234'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s90w', 'function': {'arguments': '{"query":"SELECT table_name, SUM(sales) AS total_sales FROM sales_data GROUP BY table_name ORDER BY total_sales DESC LIMIT 1"}', 'name': 'sql_db_query_checker'}, 'type': 'function'}, {'id': 'call_b3ap', 'function': {'arguments': '{"query":"SELECT table_name, SUM(sales) AS total_sales FROM sales_data GROUP BY table_name ORDER BY total_sales DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 1750, 'total_tokens': 1930, 'completion_time': 0.15, 'prompt_time': 0.239345765, 'queue_time': 0.03831731399999999, 'total_time': 0.389345765}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls',

In [18]:
response['messages'][-1].content

'I can now use the result of the query.\n\nHere is the answer to the original question:\n\nThe state with the highest sales is: None (since the query did not specify the top state)\n\nHowever, if we execute the query, we can get the results:\n\n```\nSELECT State, SUM(`iPhone Sales (in million units)` ) AS total_sales FROM AppleSales GROUP BY State ORDER BY total_sales DESC LIMIT 5\n```\n\nThis query would return the top 5 states by total sales.\n\n<tool-use>{"tool_calls":[]}</tool-use>'

In [21]:

question = "what was the apj speech ?"

response2  = agent_executor.invoke(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    
    config = config
        )

In [23]:
response2['messages'][-1].content

"The query was successful! To answer the user's question, I would simply respond with the results:\n\nState\tRegion\tiPhone Sales (in million units)\tiPad Sales (in million units)\tMac Sales (in million units)\tWearables (in million units)\tServices Revenue (in billion $)\nChongqing\tGreater China\t7.46\t6.75\t1.19\t5.88\t15.88\nGermany\tEurope\t8.63\t14.06\t7.03\t7.42\t10.12\nUK\tEurope\t5.61\t14.09\t8.78\t8.19\t19.85\nShanghai\tGreater China\t7.82\t7.97\t9.78\t2.28\t6.16\nThailand\tRest of Asia\t16.7\t8.13\t6.46\t3.48\t13.29"